In [3]:
import plotly.graph_objects as go

# Initialize the figure object
fig = go.Figure()

# Add the Scattergeo trace to mark a location
fig.add_trace(go.Scattergeo(
    # Coordinates for New York City (approx. S&P 500 location)
    lon = [-74.0060],
    lat = [40.7128],
    mode = 'markers+text', # Show both a marker and text label
    marker = dict(
        size = 10, # Marker size (5 * 2)
        color = 'red',
        # colorscale is typically used when plotting multiple points with varying Z values,
        # but kept here for consistency.
        colorscale = 'Viridis' 
    ),
    text = ['S&P 500'],
    textfont = dict(
        size = 12,
        color = "red"
    ),
    # To place the text above the marker
    textposition = "top center" 
))

# Configure the geographical settings (layout)
fig.update_geos(
    projection_type = 'orthographic', # This gives the 3D globe effect
    showcountries = True,
    showocean = True,
    oceancolor = '#a1c2fa', # Slightly nicer light blue
    countrycolor = '#333333',
    landcolor = '#e0e0e0',
    bgcolor = '#f4f4f4' # Background color for the map area
)

# Update the layout for a title and margin
fig.update_layout(
    title_text = 'S&P 500 Location (New York) on a 3D Globe',
    title_font_size = 24,
    margin={"r":0,"t":50,"l":0,"b":0}
)

# This command is crucial: it opens the interactive Plotly figure 
# in your default web browser for viewing.
fig.show()


In [4]:
import plotly.graph_objects as go
import pandas as pd # We need pandas to create the DataFrame (df) used in the plot

# --- 1. Define the missing DataFrame (df) ---
# We create sample data representing a few major global indices, 
# including a numerical value (Market_Cap_Billion) to drive the colorization.
data = {
    'Index_Name': ['S&P 500 (USA)', 'FTSE 100 (UK)', 'Nikkei 225 (Japan)', 'Hang Seng (HK)'],
    'Latitude': [40.7128, 51.5074, 35.6895, 22.3193],
    'Longitude': [-74.0060, -0.1278, 139.6917, 114.1694],
    'Market_Cap_Billion': [45000, 3500, 6000, 5000] 
}
df = pd.DataFrame(data)

# Initialize the figure object
fig = go.Figure()

# Add the Scattermapbox trace
fig.add_trace(go.Scattermapbox(
    lon = df['Longitude'],
    lat = df['Latitude'],
    mode = 'markers+text',
    
    # --- 2. Colorization Implementation ---
    marker = dict(
        # Set the size dynamically based on Market Cap
        size = df['Market_Cap_Billion'] / 500, 
        
        # Set the color dynamically based on Market Cap
        color = df['Market_Cap_Billion'],      
        
        # Choose a dynamic colorscale (Try 'Viridis', 'Plasma', or 'Electric')
        colorscale = 'Electric',               
        showscale = True,                      # Display the color bar legend
        colorbar = dict(
            title="Market Cap ($B)",
            orientation="h"
        )
    ),
    text = df['Index_Name'],
    textfont = dict(size=10, color='black'),
    textposition = "top right",
    hoverinfo = 'text',
    hovertext = df['Index_Name'] + '<br>Market Cap: $' + (df['Market_Cap_Billion']).astype(str) + 'B'
))

# Configure the mapbox layout
fig.update_layout(
    title_text = 'Global Stock Market Indices by Market Cap (Mapbox)',
    mapbox = dict(
        style = 'open-street-map', # FREE, no API key required for this style!
        zoom = 1.2,
        center = dict(lon=20, lat=30)
    ),
    margin={"r":0,"t":50,"l":0,"b":0}
)

# This command is crucial: it opens the interactive Plotly figure 
# in your default web browser for viewing.
fig.show()


In [3]:
import plotly.express as px
import dash
from dash import dcc, html
import pandas as pd

# --- Data Loading and Figure Generation ---

# Sample data: Gapminder (can be replaced with a financial dataset)
# The Gapminder dataset is included with Plotly Express and is excellent for demonstration.
df = px.data.gapminder()

# Create interactive animated bubble chart with Plotly Express
fig = px.scatter(
    df,
    x="gdpPercap",
    y="lifeExp",
    size="pop",
    color="continent",
    hover_name="country",
    log_x=True,
    size_max=60,
    animation_frame="year",        # Animate by year
    animation_group="country",     # Group animations by country
    range_x=[100,100000],
    range_y=[25,90],
    title="Global Development Over Time (Animated)",
)

# Customize layout for better appearance
fig.update_layout(
    margin={"r":20,"t":40,"l":20,"b":20},
    title_font_size=20,
    plot_bgcolor='#f9f9f9',
    paper_bgcolor='#ffffff',
    # Adjust slider button colors
    updatemenus=[{
        'buttons': [{
            'args': [None, {'frame': {'duration': 500, 'redraw': False}, 'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
            'label': 'Play',
            'method': 'animate'
        }],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }]
)

# --- Dash App Layout ---

# Initialize the Dash application
# Note: Changed _name_ to __name__ to be runnable
app = dash.Dash(__name__)

app.layout = html.Div(
    style={'maxWidth': '1000px', 'margin': '0 auto', 'padding': '20px', 'fontFamily': 'Arial, sans-serif'},
    children=[
        html.H1(
            "Interactive Data Explorer: GDP, Life Expectancy, and Population", 
            style={'textAlign': 'center', 'color': '#1f2937'}
        ),
        html.Div(
            "An animated visualization showing how key metrics have changed globally since 1952.",
            style={'textAlign': 'center', 'marginBottom': '30px', 'color': '#4b5563'}
        ),
        dcc.Graph(
            id='animated-chart',
            figure=fig,
            style={'boxShadow': '0 4px 6px rgba(0, 0, 0, 0.1)', 'borderRadius': '8px'}
        )
    ]
)

# --- Run the App ---
if __name__ == '__main__':
    # Changed _name_ and _main_ to use the correct Python dunder methods
    app.run_server(debug=True)
